In [19]:
!pip install pytesseract
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,807 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [20]:
!sudo apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [21]:
from google.colab import files

# Upload .rar file
uploaded = files.upload()

Saving scientific_publication.rar to scientific_publication.rar


In [22]:
#Extracting images from the uploaded .rar folder
!unrar x "scientific_publication.rar"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from scientific_publication.rar

Creating    scientific_publication                                    OK
Extracting  scientific_publication/doc_000016.png                          1%  OK 
Extracting  scientific_publication/doc_000045.png                          4%  OK 
Extracting  scientific_publication/doc_000050.png                          6%  OK 
Extracting  scientific_publication/doc_000061.png                          7%  OK 
Extracting  scientific_publication/doc_000089.png                          9%  OK 
Extracting  scientific_publication/doc_000121.png                         11%  OK 
Extracting  scientific_publication/doc_000128.png                         13%  OK 
Extracting  scientific_publication/doc_000130.png                         15%  OK 
Extracting  scientific_publication/doc_000171.png                      

In [58]:
import os
import pytesseract
from PIL import Image
import pandas as pd

In [59]:
def extract_text_from_image(image_path):
    # Open the image
    img = Image.open(image_path)

    # Perform OCR to extract text
    text = pytesseract.image_to_string(img)

    return text


In [60]:
image_dir = '/content/scientific_publication/'

In [61]:
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

In [62]:
image_texts = {}

for image_file in image_files:
    text = extract_text_from_image(image_file)
    image_texts[os.path.basename(image_file)] = text


In [63]:
 for image_file, text in image_texts.items():
    print(f"Text extracted from {image_file}:\n{text}\n")


Text extracted from doc_000891.png:
Reprinted from Biochemistry, 198, 27, 40
— copyright © 198 bythe American Choma! Soest and reprinted By peviniaon of copyright owner.

Native Structure and Physical Properties of Bovine Brain Kinesin and
Identification of the ATP-Binding Subunit Polypeptide!

George S. Bloom," Mark C. Wagner, K. Kevin Pfister, and Soot T- Brady

Department of Cell Biology ond Anatomy, Unversity of Texas Souwen, “Medical Center at Dallas, $323 Harry Hines
‘Boulevard, Dallas, Texas 75235

‘Received June 2, 1987; Revised Manuscript Received October 29, 1987

 

 

anstract: Kinesin was extensively purified from bovine brain cytosol by 8 ricrotubule-binding step in
{fe presence of 5-adenylylimidodipbosphate (AMP-PNP),folowat by gel filtration chromatography and
the present on ultracentrifugation. The prodets consistently contained 12400 (124K) and 64000 (64K)
aaa rides, These tw popes appear to represent heat and BN chains of kinesin, respectively,
Galton Po aiid on suer

In [64]:
!pip install datasets evaluate transformers[sentencepiece]

In [65]:
from transformers import pipeline

In [66]:
import pandas as pd

In [67]:


def extract_title_author_journal_year(text):
    # Initialize QA pipeline
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

    # Define questions
    questions = {
        "title": "What is the title of this document?",
        "author": "Who are the authors of this document?",
        "journal": "Which journal is this document published in?",
        "year": "In which year was this document published?"
    }

    # Extract information
    results = {}
    for key, question in questions.items():
        result = qa_pipeline(question=question, context=text)
        results[key] = result['answer']

    # Create DataFrame
    df = pd.DataFrame([results])

    return df


df = extract_title_author_journal_year(text)
print(df)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                               title                    author       journal  \
0  Endoerinlogy 128% 1167\n1178, 199  Dr. Vasilis Papadopoulos  Endoerinlogy   

   year  
0  1990  


In [43]:
from google.colab import files  # For file upload
import os
import re
import fitz  # PyMuPDF for PDF handling
from tika import parser  # Tika for various file format parsing
import spacy

nlp = spacy.load("en_core_web_sm")  # Load spaCy pre-trained model for English


In [44]:
def upload_file():
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        filepath = os.path.join("content", filename)  # Save to Colab's working directory
        with open(filepath, "wb") as f:
            f.write(uploaded[filename])
        return filepath
    else:
        print("No file uploaded. Please try again.")
        return None


In [69]:
def extract_metadata(filepath):


    metadata = {}


    # PyMuPDF for PDF-specific extraction
    if filepath.endswith(".pdf") and not metadata:
        with fitz.open(filepath) as doc:
                info = doc.metadata
                metadata.update({
                    "Title": info.get("title"),
                    "Year": re.search(r"\d{4}", info.get("author", "")),  # Extract year from author (heuristic)
                    "Journal": info.get("producer"),
                    "Authors": info.get("author"),
                })


    return metadata
